# Using Dask to split our data into training, validation and test sets

## Setting up local Dask compute cluster in order to speed up calculations

In [1]:
from dask.distributed import Client, LocalCluster

In [3]:
# Start a local cluster
cluster = LocalCluster(n_workers=16, threads_per_worker=1)
client = Client(cluster)

/home/kristiandampedersen/.local/share/virtualenvs/fakenewsproject-Oj_5wi4E/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33305 instead
  warnings.warn(


In [20]:
cluster.scale(16)

Task exception was never retrieved
future: <Task finished name='Task-1892675' coro=<Client._gather.<locals>.wait() done, defined at /home/kristiandampedersen/.local/share/virtualenvs/fakenewsproject-Oj_5wi4E/lib/python3.10/site-packages/distributed/client.py:2133> exception=AllExit()>
Traceback (most recent call last):
  File "/home/kristiandampedersen/.local/share/virtualenvs/fakenewsproject-Oj_5wi4E/lib/python3.10/site-packages/distributed/client.py", line 2142, in wait
    raise AllExit()
distributed.client.AllExit


## Importing the data using Dask

In [61]:
import dask.dataframe as dd
ddf = dd.read_parquet('data/chunk_114.parquet')
ddf = ddf.repartition(npartitions=10000)

In [62]:
ddf

,id,content,type,title,authors,domain,url
npartitions=10000,,,,,,,
,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [63]:
ddf.partitions[0].compute()

2023-03-07 21:26:51,626 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 756.14 MiB -- Worker memory limit: 0.97 GiB
2023-03-07 21:26:51,884 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 831.16 MiB -- Worker memory limit: 0.97 GiB
2023-03-07 21:26:51,968 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:42057 (pid=419478) exceeded 95% memory budget. Restarting...
2023-03-07 21:26:52,227 - distributed.nanny - WARNING - Restarting worker
2023-03-07 21:26:53,956 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released

KeyboardInterrupt: 

2023-03-07 21:26:59,614 - distributed.nanny - WARNING - Restarting worker


In [31]:
from dask_ml.model_selection import train_test_split
train, val = train_test_split(ddf, test_size=0.2, random_state=1, shuffle=True)

In [32]:
train.get_partition(0).compute()

2023-03-07 21:11:07,079 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 708.64 MiB -- Worker memory limit: 0.97 GiB
2023-03-07 21:11:07,441 - distributed.worker.memory - WARNING - Worker is at 86% memory usage. Pausing worker.  Process memory: 857.14 MiB -- Worker memory limit: 0.97 GiB
2023-03-07 21:11:07,481 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:35097 (pid=416213) exceeded 95% memory budget. Restarting...
2023-03-07 21:11:07,660 - distributed.nanny - WARNING - Restarting worker
2023-03-07 21:11:09,205 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released

KeyboardInterrupt: 

2023-03-07 21:11:12,322 - distributed.nanny - WARNING - Restarting worker
2023-03-07 21:11:13,477 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 695.84 MiB -- Worker memory limit: 0.97 GiB
2023-03-07 21:11:13,622 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 823.97 MiB -- Worker memory limit: 0.97 GiB
2023-03-07 21:11:13,732 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:43559 (pid=416260) exceeded 95% memory budget. Restarting...
2023-03-07 21:11:13,894 - distributed.nanny - WARNING - Restarting worker
